In [61]:
#####################################################################################
run_models = True
run_k_fold = False
overleaf_metrics = False
print_all = False
break_window_features = True
is_deadline = False
k_fold_n_splits = 5
#####################################################################################








#####################################################################################
two_labels = False
#####################################################################################


#####################################################################################
arousal_signals = ['BR']
# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR_BR']
# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'HR_BR', 'PP_BR', 'PP_HR_BR']
#####################################################################################



#####################################################################################
%run us_common_functions.ipynb
#####################################################################################


#####################################################################################
# ['dnn', 'lstm', 'lstm_1']
model_name = 'lstm_1'
#####################################################################################





#####################################################################################
# All Features - Without Deadline Study
####################################################################################
four_studies = [sim1, sim2, tt1, office_tasks]
study_combinations, all_study_combinations = get_study_combinations(four_studies)
study_combinations_to_run = all_study_combinations


### study_combinations_to_run = all_study_combinations = all_study_combinations[:1]
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
### print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################










####################################################################################
# epochs = 20 if model_name == 'lstm' else 100
epochs = 20 if 'lstm' in model_name else 100
batch_size = 32
####################################################################################


In [62]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)




###################################################
# data_6_rolling_w15_10_scaled
# data_6_rolling_w30_10_scaled
dnn_data_file = 'data_6_rolling_w15_10_scaled.csv'

dnn_window_size = dnn_data_file[-16:-11]
dnn_window_size
###################################################




# ###################################################
# # data_6_rolling_w15_5_scaled.csv
# # data_6_rolling_w30_5_scaled.csv
# # data_6_rolling_w60_5_scaled.csv
# dnn_data_file = 'data_6_rolling_w30_5_scaled.csv'

# dnn_window_size = dnn_data_file[-15:-11]
# dnn_window_size
# ####################################################################################

'30_5'

In [63]:
all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file)
# all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file).iloc[:2000,]
all_studies_df.rename({'Perinasal_Window': 'PP_Window',
                       'Heart_Window': 'HR_Window',
                       'Breathing_Window': 'BR_Window',
                      }, axis=1, inplace=True)
all_studies_df.shape

(205110, 36)

In [64]:
all_studies_df.head(2)

,Study_Name,Subject,Study_Subject,Treatment,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,PP_Mean,PP_Median,PP_SD,PP_SS,PP_Window,HR_Mean,HR_Median,HR_SD,HR_SS,HR_Window,BR_Mean,BR_Median,BR_SD,BR_SS,BR_Window,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Gender,Gender_Male
0,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-0.704197,-0.043052,0.102389,0.064272,"0.0355228579351853, 0.0403412260359834, 0.0419...",0.919132,3.768182,-0.513985,518.068554,"3.5681818181818272, 4.968181818181819, 5.56818...",-1.088477,-4.366682,0.178627,607.649078,"-5.666681818181818, -5.586681818181818, -5.546...",non-arousal,neutral,non-arousal,non-arousal,non-arousal,neutral,non-arousal,0.691472,0
1,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-0.864359,-0.053195,-0.021171,0.112911,"0.0245498539848414, 0.0139063928086793, 0.0025...",0.698460,3.568182,0.052955,400.832190,"3.76818181818183, 2.5681818181818272, 2.168181...",-0.948112,-3.966682,0.281929,486.463558,"-5.386681818181819, -5.346681818181819, -5.286...",non-arousal,neutral,non-arousal,neutral,non-arousal,neutral,neutral,0.691472,0


In [65]:
all_studies_df.columns

Index(['Study_Name', 'Subject', 'Study_Subject', 'Treatment', 'Age',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Mental', 'NASA_Performance',
       'NASA_Physical', 'NASA_Temporal', 'NASA_Total', 'PP_Mean', 'PP_Median',
       'PP_SD', 'PP_SS', 'PP_Window', 'HR_Mean', 'HR_Median', 'HR_SD', 'HR_SS',
       'HR_Window', 'BR_Mean', 'BR_Median', 'BR_SD', 'BR_SS', 'BR_Window',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Gender', 'Gender_Male'],
      dtype='object')

In [66]:
# if model_name == 'lstm':
if 'lstm' in model_name:
    all_studies_df = process_ts_features(all_studies_df)
    
    for col in model_features_ts:
        for i in range(len(all_studies_df[col][0])):
            feature_name = col[:3] + str(int(dnn_window_size[:2])-i)
            dnn_features.append(feature_name)
            all_studies_df[feature_name] = [window[i] for window in all_studies_df[col]]

    # print(list(all_studies_df.columns))

In [67]:
all_studies_df = create_categorical_labels(all_studies_df)

In [68]:
print(len(dnn_features))


if model_name == 'dnn':
    dnn_features = [x for x in dnn_features.copy() if x not in model_features_ts]


print(len(dnn_features))
# print(dnn_features)

106
106


In [69]:
# print(arousal_cat_labels)
# print(arousal_cat_labels[1])

In [70]:
start_time = time.time()

# study_combinations_to_run
# all_study_combinations
for study_combination in study_combinations_to_run:   
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    # print(all_studies_df.shape, train_df.shape)
    
    model_metrics = {}

    
    for arousal_signal in arousal_signals:
        
        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        if two_labels: 
            train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != arousal_cat_labels[1]]
        
        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
            # print(train_df.shape)
            
            model = get_trained_dnn_model(train_df, arousal_signal)


            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):
              
                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)
                    
                    if test_study_str in all_study_combinations:
                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]
                        if two_labels:
                            test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != arousal_cat_labels[1]]
                            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            # print(test_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            
                        test_dnn_model(model, test_study_str.upper(), test_df, arousal_signal)

                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics, k_fold=False), ignore_index = True)
    
        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics
    
    
display(metrics_df)
metrics_file_name = model_name+'_deadline' if is_deadline else model_name
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + '_' + dnn_window_size + '_v4_' + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------


------------------------ Train Study ------------------------
Train Data - True --> 
Counter({1: 4152, 0: 4042, 2: 3974})

Validation Data - True --> 
Counter({1: 820, 2: 812, 0: 802})

Validation Data - Prediction --> 
Counter({1: 863, 0: 805, 2: 766})

------------------------ Test Study: SIM2 ------------------------
Test Data - True --> 
Counter({2: 1282, 1: 1280, 0: 1209})

Test Data - Prediction --> 
Counter({2: 1510, 0: 1312, 1: 949})

------------------------ Test Study: TT1 ------------------------
Test Data - True --> 
Counter({1: 585, 2: 569, 0: 542})

Test Data - Prediction --> 
Counter({2: 721, 1: 499, 0: 476})

------------------------ Test Study: OFFICE_TASKS ------------------------
Test Data - True --> 
Counter({1: 8574, 0:

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity,Test_FPR,Test_ROC_AUC,Test_TPR
0,lstm,PP,sim1,sim2,0.771159,0.758154,0.920148,0.919357,0.772458,0.748832,0.771159,0.758154,0.774534,0.75352,0.0,0.0,0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.000390320...,0.9523270372027643; 0.8520298073063028; 0.9531...,0.0; 0.0008271298593879239; 0.0181968569065343...
1,lstm,PP,sim1,tt1,0.771159,0.625,0.920148,0.831673,0.772458,0.621388,0.771159,0.625,0.774534,0.628205,0.0,0.0,0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.00086655112651...,0.8942919541847065; 0.7562833206397563; 0.8431...,0.0; 0.0018450184501845018; 0.0055350553505535...
2,lstm,PP,sim1,office_tasks,0.771159,0.831454,0.920148,0.953419,0.772458,0.828451,0.771159,0.831454,0.774534,0.830695,0.0,0.0,0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0...,0.9782902200027244; 0.9087674971425693; 0.9731...,0.0; 0.00011970313622216902; 0.000598515681110...
3,lstm,HR,sim1,sim2,0.847165,0.707505,0.960041,0.881048,0.847900,0.697049,0.847165,0.707505,0.850277,0.702547,0.0,0.0,0.0; 0.0; 0.0; 0.0007833920877399138; 0.000783...,0.9158056107081595; 0.7918839774884687; 0.9348...,0.0; 0.0008210180623973727; 0.0024630541871921...
4,lstm,HR,sim1,tt1,0.847165,0.70283,0.960041,0.876784,0.847900,0.701294,0.847165,0.70283,0.850277,0.700975,0.0,0.0,0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.000888099...,0.9114128571873734; 0.8219506925606741; 0.8957...,0.0; 0.0017543859649122807; 0.0964912280701754...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,lstm,BR,sim1___sim2___tt1___office_tasks,None,0.925559,None,0.988522,None,0.925874,None,0.925559,None,0.926818,None,0.0,None,None,None,None
143,lstm,PP_HR,sim1___sim2___tt1___office_tasks,None,0.839119,None,0.953445,None,0.838623,None,0.839119,None,0.840594,None,0.0,None,None,None,None
144,lstm,HR_BR,sim1___sim2___tt1___office_tasks,None,0.850652,None,0.961011,None,0.850366,None,0.850652,None,0.852375,None,0.0,None,None,None,None
145,lstm,PP_BR,sim1___sim2___tt1___office_tasks,None,0.897367,None,0.979778,None,0.897513,None,0.897367,None,0.899508,None,0.0,None,None,None,None


In [71]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))

--- Total Time 108.0 mins (1.0 hours) ---


In [72]:
# root_path = add_path(all_studies_data_dir, metrics_dir) 
# files = ['metrics_lstm_v0.csv', 'metrics_lstm_v1.csv']
# final_file_name = 'metrics_lstm_all.csv'

# concat_df(root_path, files, final_file_name)